<a href="https://colab.research.google.com/github/hanhanwu/Hanhan_LangGraph_Exercise/blob/main/RAG_Chatbot/try_langsmith_model_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## About

* Compare 2 LLM solutions' performance
* Performance evaluation uses a testset with questions and their correct answer
* See comparison on dashboard

In [ ]:
%%capture --no-stderr
%pip install -U --quiet langsmith

In [ ]:
import pandas as pd
import os
from google.colab import userdata
from langsmith import Client
client = Client()

def _set_env(key_name: str, key_value: str):
    if key_name not in os.environ:
      os.environ[key_name] = key_value

# load the environment variables set in colab
_set_env('OPENAI_API_KEY', userdata.get('OPENAI_API_KEY'))
_set_env('LANGSMITH_API_KEY', userdata.get('LANGSMITH_API_KEY'))

## Import Testet to LangSmith

* Testset is saved [here][1]

[1]:https://github.com/hanhanwu/Hanhan_LangGraph_Exercise/blob/main/RAG_Chatbot/model_eval_testset.csv

In [ ]:
testset_df = pd.read_csv("model_eval_testset.csv", usecols=['question_id', 'question', 'answer'])
print(testset_df.shape)
display(testset_df)

(7, 3)


,question_id,question,answer
0,1,What are the battery types for electric wheelc...,"Lithium-ion battery, Dry batteries, Shielded b..."
1,2,Does aisle seats have movable armrests?,Some aisle seats have movable armrests for eas...
2,3,What is escort?,"If any of the following applies to you, you ar..."
3,4,What documents to bring for guide dogs?,We will check the guide dog vest or harness (w...
4,5,Does escort include guide dogs?,Guide dogs are considered as escort.
5,6,Is there staff available to guide visually dis...,Our staff will escort you at the airport and i...
6,7,Can I request a pre-boarding service to settle...,Pre-boarding service is available for passenge...
